In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer

In [4]:
import PIL.Image as Image
import cv2
import os

In [5]:
data_source = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
flowers_data = tf.keras.utils.get_file('flower_photos',origin=data_source,cache_dir='.',untar=True)

In [6]:
import pathlib

In [7]:
flowers_data = pathlib.Path(flowers_data)

In [8]:
len(list(flowers_data.glob('*/*.jpg')))

3670

In [9]:
flower_images_dict = {
    'dandelion': list(flowers_data.glob('dandelion/*')),
    'roses': list(flowers_data.glob('roses/*')),
    'tulips': list(flowers_data.glob('tulips/*')),
    'sunflowers': list(flowers_data.glob('sunflowers/*')),
    'daisy': list(flowers_data.glob('daisy/*')),
}

In [10]:
flower_dict_labels = {
    'dandelion':0,
    'roses':1,
    'tulips':2,
    'sunflowers':3,
    'daisy':4
}

In [11]:
x,y = [],[]

for flower_name,images in flower_images_dict.items():
  for image in images:
    img = cv2.imread(str(image))
    resized_image = cv2.resize(img,(224,224))
    x.append(resized_image)
    y.append(flower_dict_labels[flower_name])

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x = np.array(x)
y = np.array(y) 

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [15]:
x_train_scaled = np.divide(x_train,255)
x_test_scaled = np.divide(x_test,255)

In [16]:
class Conv2D(layers.Layer):

    def __init__(self,kernel_size,padding,strides,filters,activation):
        super().__init__()
        self.conv = layers.Conv2D(
            kernel_size=kernel_size,padding=padding,
            strides=strides,filters=filters,activation=activation
        )
        self.batch_norm = layers.BatchNormalization()

    def call(self,input_tensor,training=True):
        x = self.conv(input_tensor)
        x = self.batch_norm(x,training)
        return x 

In [17]:
class ResNetBlock(layers.Layer):

    def __init__(self,channels,stride=1):
        super().__init__()

        self.skip_dotted = (stride != 1)
        self.conv1 = Conv2D((3,3),'same',stride,channels,'relu')
        self.conv2 = Conv2D(kernel_size=(3,3),padding='same',filters=channels,activation='relu',strides=1)
        if self.skip_dotted:
            self.conv3 = Conv2D(filters=channels,kernel_size=(1,1),strides=stride,padding='valid',activation='relu')

    
    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        if self.skip_dotted:
            input_tensor = self.conv3(input_tensor)
        x = layers.add([input_tensor, x])
        x = tf.nn.relu(x)
        return x 

In [18]:
class ResNet34(keras.Model):

    def __init__(self,num_classes=5):
        super().__init__()
        self.conv1  = Conv2D(filters=64,kernel_size=(7,7),strides=2,padding='same',activation='relu')
        self.batch_norm = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.pool1 = layers.MaxPooling2D(pool_size=(3,3),strides=2)

        self.block1_1 = ResNetBlock(64)
        self.block1_2 = ResNetBlock(64)
        self.block1_3 = ResNetBlock(64)

        self.block2_1 = ResNetBlock(128,2)
        self.block2_2 = ResNetBlock(128)
        self.block2_3 = ResNetBlock(128)
        self.block2_4 = ResNetBlock(128)
    
        self.block3_1 = ResNetBlock(256,2)
        self.block3_2 = ResNetBlock(256)
        self.block3_3 = ResNetBlock(256)
        self.block3_4 = ResNetBlock(256)
        self.block3_5 = ResNetBlock(256)
        self.block3_6 = ResNetBlock(256)

        self.block4_1 = ResNetBlock(512,2)
        self.block4_2 = ResNetBlock(512)
        self.block4_3 = ResNetBlock(512)

        self.pool2 = layers.GlobalAveragePooling2D()
        self.fully_connected1 = layers.Dense(512,activation='relu')
        self.fully_connected2 = layers.Dense(512,activation='relu')
        self.drop_out1 = layers.Dropout(0.5)
        self.drop_out2 = layers.Dropout(0.5)
        self.classifier = layers.Dense(units=num_classes,activation='softmax') 


    def call(self,input_tensor,training=True):
        x = self.conv1(input_tensor)
        x = self.batch_norm(x,training)
        x = self.relu(x)
        x = self.pool1(x)
        #64 filter section
        x = self.block1_1(x)
        x = self.block1_2(x)
        x = self.block1_3(x)

        #128 filter section
        x = self.block2_1(x)
        x = self.block2_2(x)
        x = self.block2_3(x)
        x = self.block2_4(x)

        #256 filter section
        x = self.block3_1(x)
        x = self.block3_2(x)
        x = self.block3_3(x)
        x = self.block3_4(x)
        x = self.block3_5(x)
        x = self.block3_6(x)

        #512 filter section
        x = self.block4_1(x)
        x = self.block4_2(x)
        x = self.block4_3(x)
        x = self.pool2(x)
        
        #fully connected layer
        x = self.fully_connected1(x)
        x = self.drop_out1(x)
        x = self.fully_connected2(x)
        x = self.drop_out2(x)
        x = self.classifier(x)
        return x

In [19]:
model = ResNet34(num_classes=5)

In [20]:
model.compile(
    optimizer='adam',
    metrics =['accuracy'],
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [21]:
model.fit(x_train_scaled,y_train,epochs=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


92/92 [==============================] - 32s 195ms/step - loss: 2.2182 - accuracy: 0.2139


In [22]:
model.summary()

Model: "res_net34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  9728      
                                                                 
 batch_normalization_1 (Batc  multiple                 256       
 hNormalization)                                                 
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 res_net_block (ResNetBlock)  multiple                 74368     
                                                                 
 res_net_block_1 (ResNetBloc  multiple                 74